### Import needed packages

In [1103]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup

### Load all the necessary data

In [1104]:
#Load the country-ISO code dataset
country = pd.read_excel('../original_data/country.xlsx')

# Load the population by country dataset
pop = pd.read_csv("../original_data/Population.csv")

#Load the climate by country dataset
clim = pd.read_csv("../original_data/Climate.csv")

#Load the currencies by country dataset
curr = pd.read_csv("../original_data/Currencies.csv")

#Load the spoken languages by country dataset
lang = pd.read_csv("../original_data/Languages.csv", encoding='Latin-1')
#Load the religion by country dataset
rel = pd.read_csv("../original_data/Religion.csv")

#Load the peace index dataset
peace = pd.read_csv("../original_data/Peace_Index.csv")

#Load Wef Travel Index data
wef = pd.read_excel('../original_data/WEF_TTDI.xlsx', header=[0, 1], sheet_name='Index Performance')

# Load average restaurant and accomodation cost data
avg_rest_hot_p = pd.read_csv("../original_data/Avg_rest_hot_prices.csv")

# Load LGBTQ dataset
lgbtq = pd.read_excel("../original_data/LGBTQ .xlsx")

#### Get Cuisine Rank data by web scraping

In [1105]:
URL = "https://www.tasteatlas.com/best/cuisines?fbclid=IwAR1CFukbqGEObPMECI1SdpO_dOzeMmBjhGXvRlW8GS63JwpqUAi_0QCl4nU"
page = requests.get(URL)

In [1106]:
soup = BeautifulSoup(page.content, "html.parser")

In [1107]:
# get the block that contains the ranks
results = soup.find(id="BestCuisines")

In [1108]:
# get a list containing the names of the countries in html format
countries = results.find_all("div", class_="top-container")
# get a list containing the ratings of the countries in html format
ratings = results.find_all("div", class_="rating with-title")

In [1109]:
# extract text only from the countries html list
country_ls = []
for c in countries:
    country_ls.append(c.find('a', href=True)['href'])
print(country_ls)

['italy', 'greece', 'spain', 'japan', 'india', 'mexico', 'turkiye', 'USA', 'france', 'peru', 'china', 'brazil', 'portugal', 'poland', 'germany', 'indonesia', 'croatia', 'argentina', 'korea', 'vietnam', 'hungary', 'romania', 'philippines', 'iran', 'serbia', 'georgia', 'czech-republic', 'bulgaria', 'england', 'thailand', 'belgium', 'netherlands', 'austria', 'algeria', 'denmark', 'south-africa', 'syria', 'bih', 'malaysia', 'lebanon', 'ukraine', 'palestine', 'bangladesh', 'lithuania', 'taiwan', 'paraguay', 'pakistan', 'tunisia', 'uruguay', 'slovakia', 'egypt', 'singapore', 'afghanistan', 'ecuador', 'ethiopia', 'belarus', 'haiti', 'russia', 'north-macedonia', 'cuba', 'sri-lanka', 'sweden', 'chile', 'jamaica', 'slovenia', 'bolivia', 'venezuela', 'albania', 'northern-ireland', 'nigeria', 'colombia', 'finland', 'ireland', 'cyprus', 'estonia', 'new-zealand', 'guatemala', 'el-salvador', 'trinidad-and-tobago', 'wales', 'israel', 'azerbaijan', 'honduras', 'costa-rica', 'saudi-arabia', 'malta', 'sw

In [1110]:
# extract ratings only from the ratings html list
rating_ls = []
for rating in ratings:
    # print(box)
    rating_ls.append(float(rating.find("span").text))
print(rating_ls)

[4.72, 4.69, 4.59, 4.59, 4.54, 4.53, 4.52, 4.51, 4.51, 4.51, 4.49, 4.49, 4.47, 4.44, 4.37, 4.37, 4.33, 4.33, 4.31, 4.31, 4.26, 4.25, 4.25, 4.23, 4.23, 4.23, 4.21, 4.2, 4.18, 4.16, 4.14, 4.1, 4.09, 4.07, 4.05, 4.03, 4.03, 3.99, 3.99, 3.99, 3.98, 3.98, 3.97, 3.96, 3.96, 3.96, 3.95, 3.95, 3.95, 3.94, 3.94, 3.94, 3.94, 3.93, 3.93, 3.93, 3.93, 3.92, 3.92, 3.92, 3.92, 3.91, 3.91, 3.91, 3.91, 3.91, 3.91, 3.91, 3.91, 3.91, 3.9, 3.9, 3.9, 3.9, 3.9, 3.89, 3.89, 3.88, 3.88, 3.88, 3.85, 3.85, 3.85, 3.85, 3.85, 3.84, 3.82, 3.82, 3.8, 3.8, 3.8, 3.79, 3.79, 3.69, 3.58]


In [1111]:
# check if the country list and rating list length matches
len(rating_ls)==len(country_ls)

True

In [1112]:
# Create Data Frame with extracted Data
cuisine_rank = pd.DataFrame({'Country': country_ls, 'Ratings': rating_ls})

## EDA and Data Cleaning, Transforming

#### Country - ISO CODE Dataset

In [1113]:
country.head()

,Name,Iso3,Continent,SubContinent
0,Aruba,ABW,Latin America and the Caribbean,Caribbean
1,Andorra,ADO,Europe,Southern Europe
2,Afghanistan,AFG,"South, East and South-Eastern Asia",Southern Asia
3,Angola,AGO,Africa,Middle Africa
4,Albania,ALB,Europe,Southern Europe


In [1114]:
country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          327 non-null    object
 1   Iso3          327 non-null    object
 2   Continent     327 non-null    object
 3   SubContinent  327 non-null    object
dtypes: object(4)
memory usage: 10.3+ KB


In [1115]:
country['Name'] = country['Name'].str.title()
country['Name'] = country['Name'].str.strip()

#### Population dataset

In [1116]:
pop.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54208.0,55434.0,56234.0,56699.0,57029.0,57357.0,...,102565.0,103165.0,103776.0,104339.0,104865.0,105361.0,105846.0,106310.0,106766.0,107195.0
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130836765.0,134159786.0,137614644.0,141202036.0,144920186.0,148769974.0,...,547482863.0,562601578.0,578075373.0,593871847.0,609978946.0,626392880.0,643090131.0,660046272.0,677243299.0,694665117.0
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996967.0,9169406.0,9351442.0,9543200.0,9744772.0,9956318.0,...,31161378.0,32269592.0,33370804.0,34413603.0,35383028.0,36296111.0,37171922.0,38041757.0,38928341.0,39835428.0
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,96396419.0,98407221.0,100506960.0,102691339.0,104953470.0,107289875.0,...,370243017.0,380437896.0,390882979.0,401586651.0,412551299.0,423769930.0,435229381.0,446911598.0,458803476.0,470898870.0
4,Angola,AGO,"Population, total",SP.POP.TOTL,5454938.0,5531451.0,5608499.0,5679409.0,5734995.0,5770573.0,...,25107925.0,26015786.0,26941773.0,27884380.0,28842482.0,29816769.0,30809787.0,31825299.0,32866268.0,33933611.0


In [1117]:
pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 66 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    266 non-null    object 
 1   Country Code    266 non-null    object 
 2   Indicator Name  266 non-null    object 
 3   Indicator Code  266 non-null    object 
 4   1960            264 non-null    float64
 5   1961            264 non-null    float64
 6   1962            264 non-null    float64
 7   1963            264 non-null    float64
 8   1964            264 non-null    float64
 9   1965            264 non-null    float64
 10  1966            264 non-null    float64
 11  1967            264 non-null    float64
 12  1968            264 non-null    float64
 13  1969            264 non-null    float64
 14  1970            264 non-null    float64
 15  1971            264 non-null    float64
 16  1972            264 non-null    float64
 17  1973            264 non-null    flo

In [1118]:
pop.isna().sum()

Country Name      0
Country Code      0
Indicator Name    0
Indicator Code    0
1960              2
                 ..
2017              2
2018              2
2019              2
2020              2
2021              2
Length: 66, dtype: int64

In [1119]:
pop[pop[["2021"]].isnull().any(axis=1)]

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
69,Eritrea,ERI,"Population, total",SP.POP.TOTL,1007586.0,1033320.0,1060489.0,1088859.0,1118152.0,1148188.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,Not classified,INX,"Population, total",SP.POP.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Given the exploration done in the pop dataset, we can conclude that for the next step in the cleaning and transformations, we wish to keep only the "Country Name", " Country Code", and the most recent year 2021 (which we will rename to pop_2021). Moreover, to make sure that the dataset is homogeneous we will capitalize the first letter of each word in the "Country Name" column. We can also see that there is two null values in 2021, which will be handled.

In [1120]:
# Select the columns we are interest in
pop = pop[["Country Name", "Country Code", "2021"]]

In [1121]:
# Rename the columns
pop = pop.rename(columns = {"2021": "Pop_2021", "Country Name": "Country"})

In [1122]:
# Capitalise the first letter of each word in the Country column, and remove any extra spaces
pop["Country"] = pop["Country"].str.title()
pop["Country"] = pop["Country"].str.strip()

In [1123]:
# Remove empty rows
pop = pop.dropna()

#### Climate dataset

In [1124]:
clim.head()

,COUNTRY,DESCRIPTION
0,Afghanistan,arid to semiarid; cold winters and hot summers
1,Akrotiri,"temperate; Mediterranean with hot, dry summers..."
2,Albania,"mild temperate; cool, cloudy, wet winters; hot..."
3,Algeria,"arid to semiarid; mild, wet winters with hot, ..."
4,American Samoa,"tropical marine, moderated by southeast trade ..."


In [1125]:
clim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277 entries, 0 to 276
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   COUNTRY      277 non-null    object
 1   DESCRIPTION  277 non-null    object
dtypes: object(2)
memory usage: 4.5+ KB


In [1126]:
clim.isna().sum()

COUNTRY        0
DESCRIPTION    0
dtype: int64

Given the exploration done in the clim dataset, we can conclude that for the next step in the cleaning and transformations, we wish to keep all the columns, but rename them. Moreover, to make sure that the dataset is homogeneous we will capitalize the first letter of each word in the "COUNTRY" and "DESCRIPTION" column. We can also see that there is no null values, hence no handling of missing values is needed in this case.

In [1127]:
# Rename the columns
clim = clim.rename(columns = {"COUNTRY": "Country", "DESCRIPTION": "Climate"})

In [1128]:
# Capitalise the first letter of each word in the Country and Climate column, and remove any extra spaces in the Country column
clim["Country"] = clim["Country"].str.title()
clim["Country"] = clim["Country"].str.strip()
clim["Climate"] = clim["Climate"].str.title()

In [1129]:
# Set a new column that classifies climates 
conditions = [(clim["Climate"].str.contains("Tropical|Arid|Semiarid|Desert")),
              (clim["Climate"].str.contains("Temperate|Continental|Hot Summers And Cold Winters" )), 
              (clim["Climate"].str.contains("Mediterranean|Subtropical")),
              (clim["Climate"].str.contains("Highland")), 
              (clim["Climate"].str.contains("Polar|Antarctic")),  
              (clim["Climate"].str.contains("Equatorial"))]

choices = ["Tropical", "Temperate", "Subtropical", "Highland", "Polar", "Equatorial" ]
clim["Climate Zone"] = np.select(conditions, choices, default = "")

#### Currency dataset

In [1130]:
curr.head()

,Entity,Currency,AlphabeticCode,NumericCode,MinorUnit,WithdrawalDate
0,AFGHANISTAN,Afghani,AFN,971.0,2,NaN
1,ÅLAND ISLANDS,Euro,EUR,978.0,2,NaN
2,ALBANIA,Lek,ALL,8.0,2,NaN
3,ALGERIA,Algerian Dinar,DZD,12.0,2,NaN
4,AMERICAN SAMOA,US Dollar,USD,840.0,2,NaN


In [1131]:
curr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441 entries, 0 to 440
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Entity          441 non-null    object 
 1   Currency        441 non-null    object 
 2   AlphabeticCode  438 non-null    object 
 3   NumericCode     435 non-null    float64
 4   MinorUnit       276 non-null    object 
 5   WithdrawalDate  162 non-null    object 
dtypes: float64(1), object(5)
memory usage: 20.8+ KB


In [1132]:
curr.isna().sum()

Entity              0
Currency            0
AlphabeticCode      3
NumericCode         6
MinorUnit         165
WithdrawalDate    279
dtype: int64

In [1133]:
curr[curr[["AlphabeticCode"]].isnull().any(axis=1)]

,Entity,Currency,AlphabeticCode,NumericCode,MinorUnit,WithdrawalDate
8,ANTARCTICA,No universal currency,NaN,NaN,NaN,NaN
183,"PALESTINE, STATE OF",No universal currency,NaN,NaN,NaN,NaN
222,SOUTH GEORGIA AND THE SOUTH SANDWICH ISLANDS,No universal currency,NaN,NaN,NaN,NaN


In [1134]:
curr[curr[["WithdrawalDate"]].notnull().any(axis=1)]

,Entity,Currency,AlphabeticCode,NumericCode,MinorUnit,WithdrawalDate
279,AFGHANISTAN,Afghani,AFA,4.0,NaN,2003-01
280,ÅLAND ISLANDS,Markka,FIM,246.0,NaN,2002-03
281,ALBANIA,Old Lek,ALK,8.0,NaN,1989-12
282,ANDORRA,Andorran Peseta,ADP,20.0,NaN,2003-07
283,ANDORRA,Spanish Peseta,ESP,724.0,NaN,2002-03
...,...,...,...,...,...,...
436,ZIMBABWE,Zimbabwe Dollar (new),ZWN,942.0,NaN,2006-09
437,ZIMBABWE,Zimbabwe Dollar,ZWR,935.0,NaN,2009-06
438,ZZ01_Gold-Franc,Gold-Franc,XFO,NaN,NaN,2006-10
439,ZZ02_RINET Funds Code,RINET Funds Code,XRE,NaN,NaN,1999-11


Given the exploration done in the curr dataset, we can conclude that for the next step in the cleaning and transformations, we wish to keep only the "Entity", "Currency", and "AlphabeticCode" columns, but rename them. Moreover, to make sure that the dataset is homogeneous we will capitalize the first letter of each word in the "Entity" and "Currency" column. We can also see that there is three null values in the "AlphabeticCode" column, which will be handled. In addition, there are a number of currencies which are no longer used, we will them subset this dataset to only contain currently accepted currencies.

In [1135]:
# Keep only the rows without a withdrawal date
curr = curr[curr[["WithdrawalDate"]].isnull().any(axis=1)]

In [1136]:
# Select the columns to keep
curr = curr[["Entity", "Currency", "AlphabeticCode"]]

In [1137]:
# Rename the columns
curr = curr.rename(columns = {"Entity": "Country", "AlphabeticCode": "Currency Code"})

In [1138]:
# Capitalise the first letter of each word in the Country and Currency columns, and remove any extra spaces in the country column. Remove any text after "(" in the country column.
curr["Country"] = curr["Country"].str.title()
curr["Country"] = curr['Country'].str.partition("(")[0]
curr["Country"] = curr["Country"].str.strip()
curr["Currency"] = curr["Currency"].str.title()

In [1139]:
# Fill the null values with Non Applicable
curr["Currency Code"].fillna("Non Applicable", inplace= True)

#### Language dataset

In [1140]:
lang.head(50)

,Country,Languages Spoken
0,Afghanistan,"Dari Persian, Pashtu (both official), other Tu..."
1,Albania,"Albanian (Tosk is the official dialect), Greek"
2,Algeria,"Arabic (official), French, Berber dialects"
3,Andorra,"CatalÃ¡n (official), French, Castilian, Portug..."
4,Angola,"Portuguese (official), Bantu and other African..."
5,Antigua and Barbuda,"English (official), local dialects"
6,Argentina,"Spanish (official), English, Italian, German, ..."
7,Armenia,"Armenian 98%, Yezidi, Russian"
8,Australia,"English 79%, native and other languages"
9,Austria,"German (official nationwide); Slovene, Croatia..."


In [1141]:
lang.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Country           198 non-null    object
 1   Languages Spoken  198 non-null    object
dtypes: object(2)
memory usage: 3.2+ KB


In [1142]:
lang.isna().sum()

Country             0
Languages Spoken    0
dtype: int64

Given the exploration done in the lang dataset, we can conclude that for the next step in the cleaning and transformations, we wish to keep all the columns, but rename them. Moreover, to make sure that the dataset is homogeneous we will capitalize the first letter of each word in the "Country" and "Language Spoken" column. We can also see that there is no null values, hence no handling of missing values is needed in this case. In addition, we can see that the languages spoken includes more than the official languages, in the cleaning we will strive to only keep the official languages.

In [1143]:
# Rename the columns
lang = lang.rename(columns = {"Languages Spoken": "Official Language"})

In [1144]:
def language_cleaning(lan: str):
    """ This function is to clean the official languages, the rule is
    - If the official contains '(official)' in the text, adopt languages that come before '(official)
    - Otherwise take only first language"""

    # replace 'and' to comma
    # delete numbers and percentage
    lan = lan.title()
    lan = lan.replace(' And ', ', ')
    lan = re.sub("(\s[\d.]+%)", "", lan)

    # first if the text contains 'official', we are going to keep all the official
    if 'Official' in lan:
        lan = lan.partition("(")[0].strip()

    # otherwise, we are going to keep first one as a main language spoken
    else:
        lan = lan.partition(",")[0].strip()
    return lan

In [1145]:
# apply language cleaning function to the 'languages Spoken' column
lang['Official Language'] = lang['Official Language'].apply(language_cleaning)

In [1146]:
# Capitalise the first letter of each word in the Country and Official Language columns
lang["Country"] = lang["Country"].str.title()
lang["Country"] = lang["Country"].str.strip()

In [1147]:
lang

,Country,Official Language
0,Afghanistan,"Dari Persian, Pashtu"
1,Albania,Albanian
2,Algeria,Arabic
3,Andorra,Catalã¡N
4,Angola,Portuguese
...,...,...
193,Vietnam,Vietnamese
194,Western Sahara (Proposed State),"Hassaniya Arabic, Moroccan Arabic"
195,Yemen,Arabic
196,Zambia,English


#### Religion dataset

In [1148]:
rel.head()

,Country or Area,Year,Area,Sex,Religion,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Albania,2011,Total,Both Sexes,Total,Census - de jure - complete tabulation,"Final figure, complete",2013.0,2800138.0,NaN
1,Albania,2011,Total,Both Sexes,Atheist,Census - de jure - complete tabulation,"Final figure, complete",2013.0,69995.0,NaN
2,Albania,2011,Total,Both Sexes,Catholic,Census - de jure - complete tabulation,"Final figure, complete",2013.0,280921.0,NaN
3,Albania,2011,Total,Both Sexes,Evangelical,Census - de jure - complete tabulation,"Final figure, complete",2013.0,3797.0,NaN
4,Albania,2011,Total,Both Sexes,Muslim,Census - de jure - complete tabulation,"Final figure, complete",2013.0,1587608.0,NaN


In [1149]:
rel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3416 entries, 0 to 3415
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Country or Area  3416 non-null   object 
 1   Year             3416 non-null   object 
 2   Area             3333 non-null   object 
 3   Sex              3332 non-null   object 
 4   Religion         3332 non-null   object 
 5   Record Type      3332 non-null   object 
 6   Reliability      3332 non-null   object 
 7   Source Year      3332 non-null   float64
 8   Value            3332 non-null   float64
 9   Value Footnotes  1505 non-null   object 
dtypes: float64(2), object(8)
memory usage: 267.0+ KB


In [1150]:
rel.isna().sum()

Country or Area       0
Year                  0
Area                 83
Sex                  84
Religion             84
Record Type          84
Reliability          84
Source Year          84
Value                84
Value Footnotes    1911
dtype: int64

In [1151]:
rel[rel[["Religion"]].isnull().any(axis=1)]

,Country or Area,Year,Area,Sex,Religion,Record Type,Reliability,Source Year,Value,Value Footnotes
3332,footnoteSeqID,Footnote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3333,1,"Because of rounding, totals are not in all cas...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3334,2,These data have been randomly rounded to prote...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3335,3,"Including population in off-shore, migratory a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3336,4,"Category 'No religion' includes Agnoticism, At...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3411,79,Data of mini-census taken in 2017.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3412,80,Data refer to resident population.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3413,81,Excluding population enumerated in hotels.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3414,82,"Other includes Orthodox, Bahai, Other Christia...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1152]:
rel[rel[["Area"]].isnull().any(axis=1)]

,Country or Area,Year,Area,Sex,Religion,Record Type,Reliability,Source Year,Value,Value Footnotes
3332,footnoteSeqID,Footnote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3333,1,"Because of rounding, totals are not in all cas...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3334,2,These data have been randomly rounded to prote...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3335,3,"Including population in off-shore, migratory a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3336,4,"Category 'No religion' includes Agnoticism, At...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3411,79,Data of mini-census taken in 2017.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3412,80,Data refer to resident population.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3413,81,Excluding population enumerated in hotels.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3414,82,"Other includes Orthodox, Bahai, Other Christia...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1153]:
rel[rel[["Sex"]].isnull().any(axis=1)]

,Country or Area,Year,Area,Sex,Religion,Record Type,Reliability,Source Year,Value,Value Footnotes
3332,footnoteSeqID,Footnote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3333,1,"Because of rounding, totals are not in all cas...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3334,2,These data have been randomly rounded to prote...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3335,3,"Including population in off-shore, migratory a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3336,4,"Category 'No religion' includes Agnoticism, At...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3411,79,Data of mini-census taken in 2017.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3412,80,Data refer to resident population.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3413,81,Excluding population enumerated in hotels.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3414,82,"Other includes Orthodox, Bahai, Other Christia...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Given the exploration done in the rel dataset, we can conclude that for the next step in the cleaning and transformations, we wish to keep the "Country or Area" and "Religion" columns but rename them. Moreover, to make sure that the dataset is homogeneous we will capitalize the first letter of each word in the "Country or Area" column. We can also see that there is multiple null values, that arise due to the existing footnotes at the end of the document, which will be dealt with. In addition, we can see that for each country there is the listing of all the religions present in the country. As we only wish to keep the main one the others will be removed.

In [1154]:
# Remove the footnote rows, which is where the Religion column 
rel = rel[rel[["Religion"]].notna().any(axis=1)]

In [1155]:
# Remove the rows with the total number of religious people. Also remove the other, not stated, and refused to answer religions
rel = rel[rel["Religion"].str.contains("Total|Other|Not Stated|Refused to answer") == False]

In [1156]:
# Keep the data from the most recent year for each country
rel = rel.sort_values(by=["Country or Area", "Year"])
rel = rel.drop_duplicates(subset=['Country or Area', "Religion"], keep='last')

In [1157]:
# Keep the rows with the highest value, meaning the main religion of each country
rel = rel.sort_values(by=["Country or Area", "Value"])
rel = rel.drop_duplicates(subset=['Country or Area'], keep='last')

In [1158]:
# Select the columns to keep
rel = rel[["Country or Area", "Religion"]]

In [1159]:
# Rename the columns
rel = rel.rename(columns = {"Country or Area": "Country", "Religion": "Main Religion"})

In [1160]:
# Capitalise the first letter of each word in the Country and religion, and remove any extra spaces in the country column.
rel["Country"] = rel["Country"].str.title()
rel["Country"] = rel["Country"].str.strip()
rel["Main Religion"] = rel["Main Religion"].str.title()

#### Peace index dataset

In [1161]:
peace.head()

,RANK,COUNTRY,SCORE
0,1.0,Iceland,1.100
1,2.0,New Zealand,1.253
2,3.0,Denmark,1.256
3,4.0,Portugal,1.267
4,5.0,Slovenia,1.315


In [1162]:
peace.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   RANK     163 non-null    float64
 1   COUNTRY  163 non-null    object 
 2   SCORE    163 non-null    float64
dtypes: float64(2), object(1)
memory usage: 4.3+ KB


In [1163]:
peace.isna().sum()

RANK       14
COUNTRY    14
SCORE      14
dtype: int64

In [1164]:
peace[peace[["COUNTRY"]].isnull().any(axis=1)]

,RANK,COUNTRY,SCORE
16,NaN,NaN,NaN
45,NaN,NaN,NaN
74,NaN,NaN,NaN
86,NaN,NaN,NaN
95,NaN,NaN,NaN
108,NaN,NaN,NaN
125,NaN,NaN,NaN
128,NaN,NaN,NaN
138,NaN,NaN,NaN
155,NaN,NaN,NaN


Given the exploration done in the peace dataset, we can conclude that for the next step in the cleaning and transformations, we wish to keep the "RANK", and "COUNTRY" columns, but rename them. Moreover, to make sure that the dataset is homogeneous we will capitalize the first letter of each word in the "COUNTRY" column. We can also see that there is multiple null values, that arise due to the existing of completely empty rows, which will be dealt with.

In [1165]:
# Select the columns to keep
peace = peace[["RANK", "COUNTRY"]]

In [1166]:
# Rename the columns
peace = peace.rename(columns = {"RANK": "Rank", "COUNTRY": "Country"})

In [1167]:
# Capitalise the first letter of each word in the Country column
peace["Country"] = peace["Country"].str.title()
peace["Country"] = peace["Country"].str.strip()


In [1168]:
# Drop null rows
peace = peace.dropna()

#### WEF Dataset

In [1169]:
# to flatten the double header from excel
wef.columns=wef.columns.to_flat_index()

In [1170]:
wef.head()

,"(Unnamed: 0_level_0, ISO Code)","(Unnamed: 1_level_0, Economy)","(Unnamed: 2_level_0, Region)","(Unnamed: 3_level_0, Sub Region)","(Unnamed: 4_level_0, Income Group)","(Travel & Tourism Development Index , 2019 Value)","(Travel & Tourism Development Index , 2019 Rank)","(Travel & Tourism Development Index , 2021 Value)","(Travel & Tourism Development Index , 2021 Rank)","(Travel & Tourism Development Index , % Dif Score)",...,"(Socioeconomic Resilience & Conditions pillar, 2021 Value)","(Socioeconomic Resilience & Conditions pillar, 2021 Rank)","(Socioeconomic Resilience & Conditions pillar, % Dif Score)","(Socioeconomic Resilience & Conditions pillar, Rank Change)","(T&T Demand Pressure & Impact pillar, 2019 Value)","(T&T Demand Pressure & Impact pillar, 2019 Rank)","(T&T Demand Pressure & Impact pillar, 2021 Value)","(T&T Demand Pressure & Impact pillar, 2021 Rank)","(T&T Demand Pressure & Impact pillar, % Dif Score)","(T&T Demand Pressure & Impact pillar, Rank Change)"
0,JPN,Japan,Asia-Pacific,Eastern Asia-Pacific,High-income economies,5.209308,2,5.245868,1,0.007018,...,5.710859,11,0.035617,6,4.162195,43,4.287192,41,0.030031,2
1,USA,United States,The Americas,North and Central America,High-income economies,5.253746,1,5.199991,2,-0.010232,...,4.496351,44,0.018056,-1,4.835106,4,4.945433,3,0.022818,1
2,ESP,Spain,Europe and Eurasia,Southern Europe,High-income economies,5.156125,5,5.154733,3,-0.000270,...,5.297103,25,0.068278,5,3.668553,89,3.768780,81,0.027321,8
3,FRA,France,Europe and Eurasia,Western Europe,High-income economies,5.139553,6,5.126821,4,-0.002477,...,5.734719,10,0.005727,1,3.447736,106,3.429899,108,-0.005174,-2
4,DEU,Germany,Europe and Eurasia,Western Europe,High-income economies,5.158769,4,5.064452,5,-0.018283,...,5.884420,8,0.012092,0,3.691413,87,3.921128,73,0.062230,14


In [1171]:
wef.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Columns: 143 entries, ('Unnamed: 0_level_0', 'ISO Code') to ('T&T Demand Pressure & Impact pillar', 'Rank Change')
dtypes: float64(69), int64(69), object(5)
memory usage: 130.8+ KB


In [1172]:
wef.isna().sum()

(Unnamed: 0_level_0, ISO Code)                        0
(Unnamed: 1_level_0, Economy)                         0
(Unnamed: 2_level_0, Region)                          0
(Unnamed: 3_level_0, Sub Region)                      0
(Unnamed: 4_level_0, Income Group)                    0
                                                     ..
(T&T Demand Pressure & Impact pillar, 2019 Rank)      0
(T&T Demand Pressure & Impact pillar, 2021 Value)     0
(T&T Demand Pressure & Impact pillar, 2021 Rank)      0
(T&T Demand Pressure & Impact pillar, % Dif Score)    0
(T&T Demand Pressure & Impact pillar, Rank Change)    0
Length: 143, dtype: int64

Among 143 columns we have in dataset, we will choose relevant columns and drop all the others. We found we have no null data in the dataset. Moreover, as the dataset have ISO code itself, there is no further cleaning needed for the country name.

In [1173]:
# Rename columns
wef.rename(columns = {('Unnamed: 0_level_0', 'ISO Code'):'ISO Code',
                      ('Unnamed: 1_level_0', 'Economy'):'Country',
                      ('Unnamed: 2_level_0', 'Region'):'Continent',
                      ('Unnamed: 3_level_0', 'Sub Region'):'Sub Continent',
                      ('Unnamed: 4_level_0', 'Income Group'):'Income Group'
                      }, inplace = True)

In [1174]:
# To check the index of the columns that we wish to drop
for i, col in enumerate(wef.columns):
    print(i, col)

0 ISO Code
1 Country
2 Continent
3 Sub Continent
4 Income Group
5 ('Travel & Tourism Development Index ', '2019 Value')
6 ('Travel & Tourism Development Index ', '2019 Rank')
7 ('Travel & Tourism Development Index ', '2021 Value')
8 ('Travel & Tourism Development Index ', '2021 Rank')
9 ('Travel & Tourism Development Index ', '% Dif Score')
10 ('Travel & Tourism Development Index ', 'Rank Change')
11 ('Enabling Environment subindex', '2019 Value')
12 ('Enabling Environment subindex', '2019 Rank')
13 ('Enabling Environment subindex', '2021 Value')
14 ('Enabling Environment subindex', '2021 Rank')
15 ('Enabling Environment subindex', '% Dif Score')
16 ('Enabling Environment subindex', 'Rank Change')
17 ('Travel and Tourism Policy and Enabling\nConditions subindex', '2019 Value')
18 ('Travel and Tourism Policy and Enabling\nConditions subindex', '2019 Rank')
19 ('Travel and Tourism Policy and Enabling\nConditions subindex', '2021 Value')
20 ('Travel and Tourism Policy and Enabling\nCondit

In [1175]:
# Drop columns from the behind to prevent index changing
wef = wef.iloc[:, [0,1,2,3,4,7,8,25,26,49,50,55,56,79,80,85,86,109,110,115,116,127,128,139,140]]
wef.columns

Index([                                           'ISO Code',
                                                   'Country',
                                                 'Continent',
                                             'Sub Continent',
                                              'Income Group',
       ('Travel & Tourism Development Index ', '2021 Value'),
        ('Travel & Tourism Development Index ', '2021 Rank'),
                   ('Infrastructure subindex', '2021 Value'),
                    ('Infrastructure subindex', '2021 Rank'),
                ('Safety and Security pillar', '2021 Value'),
                 ('Safety and Security pillar', '2021 Rank'),
                 ('Health and Hygiene pillar', '2021 Value'),
                  ('Health and Hygiene pillar', '2021 Rank'),
             ('International Openness pillar', '2021 Value'),
              ('International Openness pillar', '2021 Rank'),
              ('Price competitiveness pillar', '2021 Value'),
        

##### Set new columns that classify each index into Very good, Good, Limited

In [1176]:
# Infrastructure subindex
conditions = [(wef[ ('Infrastructure subindex', '2021 Rank')] <= 40), (wef[('Infrastructure subindex', '2021 Rank')] >= 80)]
choices = ["Very good", "Limited"]
wef["Infrastructure subindex, classification"] = np.select(conditions, choices, default = "Good")

In [1177]:
# Safety and Security pillar
conditions = [(wef[('Safety and Security pillar', '2021 Rank')] <= 40), (wef[ ('Safety and Security pillar', '2021 Rank')] >= 80)]
choices = ["Very good", "Limited"]
wef["Safety and Security, Classification"] = np.select(conditions, choices, default = "Good")

In [1178]:
# Health and Hygiene pillar
conditions = [(wef[('Health and Hygiene pillar', '2021 Rank')] <= 40), (wef[('Health and Hygiene pillar', '2021 Rank')] >= 80)]
choices = ["Very good", "Limited"]
wef["Health and Hygiene, Classification"] = np.select(conditions, choices, default = "Good")

In [1179]:
# International Openness pillar
conditions = [(wef[('International Openness pillar', '2021 Rank')] <= 40), (wef[('International Openness pillar', '2021 Rank')] >= 80)]
choices = ["Very good", "Limited"]
wef["International Openess, Classification"] = np.select(conditions, choices, default = "Good")

In [1180]:
# Price competitiveness pillar
conditions = [(wef[('Price competitiveness pillar', '2021 Rank')] <= 40), (wef[ ('Price competitiveness pillar', '2021 Rank')] >= 80)]
choices = ["Very good", "Limited"]
wef["Price competitiveness, Classification"] = np.select(conditions, choices, default = "Good")

In [1181]:
# Natural Resources pillar
conditions = [(wef[('Natural Resources pillar', '2021 Rank')] <= 40), (wef[('Natural Resources pillar', '2021 Rank')] >= 80)]
choices = ["Very good", "Limited"]
wef["Natural Resources, Classification"] = np.select(conditions, choices, default = "Good")

In [1182]:
# Cultural Resources pillar
conditions = [(wef[('Cultural Resources pillar', '2021 Rank')] <= 40), (wef[('Cultural Resources pillar', '2021 Rank')] >= 80)]
choices = ["Very good", "Limited"]
wef["Cultural Resources, Classification"] = np.select(conditions, choices, default = "Good")

In [1183]:
# Environmental Sustainability pillar
conditions = [(wef[('Environmental Sustainability pillar', '2021 Rank')] <= 40), (wef[('Environmental Sustainability pillar', '2021 Rank')] >= 80)]
choices = ["Very good", "Limited"]
wef["Environmental Sustainability, Classification"] = np.select(conditions, choices, default = "Good")

In [1184]:
# T&T Demand Pressure & Impact pillar
conditions = [(wef[('T&T Demand Pressure & Impact pillar', '2021 Rank')] <= 40), (wef[('T&T Demand Pressure & Impact pillar', '2021 Rank')] >= 80)]
choices = ["Very good", "Limited"]
wef["T&T Demand Pressure & Impact, Classification"] = np.select(conditions, choices, default = "Good")

#### Average Restaurant and Accomodation Cost Dataset

In [1185]:
avg_rest_hot_p.head()

,Country Name,Country Code,Classification Name,Classification Code,Series Name,Series Code,2017 [YR2017]
0,Albania,ALB,Price level index (World = 100),PX.WL,1111000:RESTAURANTS AND HOTELS,1111000.0,50.78426859
1,Algeria,DZA,Price level index (World = 100),PX.WL,1111000:RESTAURANTS AND HOTELS,1111000.0,33.93366644
2,American Samoa,ASM,Price level index (World = 100),PX.WL,1111000:RESTAURANTS AND HOTELS,1111000.0,..
3,Angola,AGO,Price level index (World = 100),PX.WL,1111000:RESTAURANTS AND HOTELS,1111000.0,79.22385228
4,Anguilla,AIA,Price level index (World = 100),PX.WL,1111000:RESTAURANTS AND HOTELS,1111000.0,122.29901


In [1186]:
avg_rest_hot_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country Name         224 non-null    object 
 1   Country Code         222 non-null    object 
 2   Classification Name  222 non-null    object 
 3   Classification Code  222 non-null    object 
 4   Series Name          222 non-null    object 
 5   Series Code          222 non-null    float64
 6   2017 [YR2017]        222 non-null    object 
dtypes: float64(1), object(6)
memory usage: 12.5+ KB


In [1187]:
avg_rest_hot_p.isna().sum()

Country Name           3
Country Code           5
Classification Name    5
Classification Code    5
Series Name            5
Series Code            5
2017 [YR2017]          5
dtype: int64

In [1188]:
avg_rest_hot_p[avg_rest_hot_p[["Country Name"]].isnull().any(axis=1)]

,Country Name,Country Code,Classification Name,Classification Code,Series Name,Series Code,2017 [YR2017]
222,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1189]:
avg_rest_hot_p[avg_rest_hot_p[["Country Code"]].isnull().any(axis=1)]

,Country Name,Country Code,Classification Name,Classification Code,Series Name,Series Code,2017 [YR2017]
222,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225,Data from database: ICP 2017,NaN,NaN,NaN,NaN,NaN,NaN
226,Last Updated: 10/21/2020,NaN,NaN,NaN,NaN,NaN,NaN


Given the exploration done in the avg_rest_hot_p dataset, we can conclude that for the next step in the cleaning and transformations, we wish to keep only the "Country Name", " Country Code", and the "2017 [YR2017]" columns, but we will rename them. Moreover, to make sure that the dataset is homogeneous we will capitalize the first letter of each word in the "Country Name" column. We can also see that there is 5 null values, which arise due to empty rows and footnotes, these will be handled accordingly. Additionally, we can see that some countries in the average price have ".." instead of being null, which will be solved.

In [1190]:
# Select the columns we are interest in
avg_rest_hot_p = avg_rest_hot_p[["Country Name", "Country Code", "2017 [YR2017]"]]

In [1191]:
# Rename the columns
avg_rest_hot_p = avg_rest_hot_p.rename(columns = {"Country Name": "Country", "2017 [YR2017]": "Average Hotel and Restaurant Cost"})

In [1192]:
# Capitalise the first letter of each word in the Country column, and remove any extra spaces
avg_rest_hot_p["Country"] = avg_rest_hot_p["Country"].str.title()
avg_rest_hot_p["Country"] = avg_rest_hot_p["Country"].str.strip()

In [1193]:
# Substitute ".." for null value
avg_rest_hot_p["Average Hotel and Restaurant Cost"] = avg_rest_hot_p["Average Hotel and Restaurant Cost"].replace("..","")

In [1194]:
# Remove empty rows
avg_rest_hot_p = avg_rest_hot_p.dropna()

In [1195]:
# Remove empty rows
avg_rest_hot_p = avg_rest_hot_p.dropna()

In [1196]:
# Convert the column from string to numeric value
avg_rest_hot_p["Average Hotel and Restaurant Cost"] = pd.to_numeric(avg_rest_hot_p["Average Hotel and Restaurant Cost"], errors = "coerce")

In [1197]:
# Round the numbers to two decimal points
avg_rest_hot_p["Average Hotel and Restaurant Cost"] = avg_rest_hot_p["Average Hotel and Restaurant Cost"].round(2)

In [1198]:
# Set a new column that classifies countries into budget travel, comfortable travel and luxury travel
conditions = [(avg_rest_hot_p["Average Hotel and Restaurant Cost"] <= 70), ((avg_rest_hot_p["Average Hotel and Restaurant Cost"] > 70) & (avg_rest_hot_p["Average Hotel and Restaurant Cost"] <= 150)), (avg_rest_hot_p["Average Hotel and Restaurant Cost"] > 150)]
choices = ["Budget Traveler", "Comfort traveler", "Luxury Travel"]
avg_rest_hot_p["Type Traveler"] = np.select(conditions, choices, default = "Non Applicable")

#### LGBTQ Dataset

In [1199]:
lgbtq.head()

,RANK,COUNTRY,GAI
0,1,Iceland,9.78
1,2,Netherlands,9.46
2,3,Norway,9.38
3,4,Sweden,9.18
4,5,Canada,9.02


In [1200]:
lgbtq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   RANK     175 non-null    int64  
 1   COUNTRY  175 non-null    object 
 2   GAI      175 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.2+ KB


In [1201]:
lgbtq['COUNTRY'] = lgbtq['COUNTRY'].str.title()

#### Cuisine Rank Dataset

In [1202]:
cuisine_rank.head()

,Country,Ratings
0,italy,4.72
1,greece,4.69
2,spain,4.59
3,japan,4.59
4,india,4.54


In [1203]:
cuisine_rank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  95 non-null     object 
 1   Ratings  95 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.6+ KB


In [1204]:
cuisine_rank['Country'] = cuisine_rank['Country'].str.replace('-', ' ')
cuisine_rank['Country'] = cuisine_rank['Country'].str.title()

In [1205]:
cuisine_rank

,Country,Ratings
0,Italy,4.72
1,Greece,4.69
2,Spain,4.59
3,Japan,4.59
4,India,4.54
...,...,...
90,Iceland,3.80
91,Canada,3.79
92,Latvia,3.79
93,Morocco,3.69


## Adding ISO Code to All Datasets by Joining with Country Dataset

#### Check climate - country

In [1206]:
clim = clim.merge(country[['Name','Iso3']], how='left', left_on='Country', right_on='Name')
clim.head()

,Country,Climate,Climate Zone,Name,Iso3
0,Afghanistan,Arid To Semiarid; Cold Winters And Hot Summers,Tropical,Afghanistan,AFG
1,Akrotiri,"Temperate; Mediterranean With Hot, Dry Summers...",Temperate,NaN,NaN
2,Albania,"Mild Temperate; Cool, Cloudy, Wet Winters; Hot...",Temperate,Albania,ALB
3,Algeria,"Arid To Semiarid; Mild, Wet Winters With Hot, ...",Tropical,Algeria,DZA
4,American Samoa,"Tropical Marine, Moderated By Southeast Trade ...",Tropical,American Samoa,ASM


In [1207]:
# check the countries that are not merged
clim[clim['Name'].isnull()]

,Country,Climate,Climate Zone,Name,Iso3
1,Akrotiri,"Temperate; Mediterranean With Hot, Dry Summers...",Temperate,NaN,NaN
7,Anguilla,Tropical; Moderated By Northeast Trade Winds,Tropical,NaN,NaN
8,Antarctica,"Severe Low Temperatures Vary With Latitude, El...",Polar,NaN,NaN
10,Arctic Ocean,Polar Climate Characterized By Persistent Cold...,Polar,NaN,NaN
14,Ashmore And Cartier Islands,Tropical,Tropical,NaN,NaN
...,...,...,...,...,...
270,Virgin Islands,"Subtropical, Tempered By Easterly Trade Winds,...",Subtropical,NaN,NaN
271,Wake Island,Tropical,Tropical,NaN,NaN
272,Wallis And Futuna,"Tropical; Hot, Rainy Season (November To April...",Tropical,NaN,NaN
273,West Bank,Temperate; Temperature And Precipitation Vary ...,Temperate,NaN,NaN


In [1208]:
# drop the countries that are not merged
clim = clim.dropna(subset=['Name'])
# drop the merged country name column to avoid repetition
clim = clim.drop(['Name'], axis=1)

In [1209]:
clim.head()

,Country,Climate,Climate Zone,Iso3
0,Afghanistan,Arid To Semiarid; Cold Winters And Hot Summers,Tropical,AFG
2,Albania,"Mild Temperate; Cool, Cloudy, Wet Winters; Hot...",Temperate,ALB
3,Algeria,"Arid To Semiarid; Mild, Wet Winters With Hot, ...",Tropical,DZA
4,American Samoa,"Tropical Marine, Moderated By Southeast Trade ...",Tropical,ASM
5,Andorra,"Temperate; Snowy, Cold Winters And Warm, Dry S...",Temperate,ADO


#### Check Currency - Country

In [1210]:
curr = curr.merge(country[['Name','Iso3']], how='left', left_on='Country', right_on='Name')
curr.head()

,Country,Currency,Currency Code,Name,Iso3
0,Afghanistan,Afghani,AFN,Afghanistan,AFG
1,Åland Islands,Euro,EUR,NaN,NaN
2,Albania,Lek,ALL,Albania,ALB
3,Algeria,Algerian Dinar,DZD,Algeria,DZA
4,American Samoa,Us Dollar,USD,American Samoa,ASM


In [1211]:
curr[curr['Name'].isnull()]

,Country,Currency,Currency Code,Name,Iso3
1,Åland Islands,Euro,EUR,NaN,NaN
7,Anguilla,East Caribbean Dollar,XCD,NaN,NaN
8,Antarctica,No Universal Currency,Non Applicable,NaN,NaN
29,"Bonaire, Sint Eustatius And Saba",Us Dollar,USD,NaN,NaN
32,Bouvet Island,Norwegian Krone,NOK,NaN,NaN
34,British Indian Ocean Territory,Us Dollar,USD,NaN,NaN
49,Christmas Island,Australian Dollar,AUD,NaN,NaN
50,Cocos Keeling Islands,Australian Dollar,AUD,NaN,NaN
54,The Democratic Republic Of The Congo,Congolese Franc,CDF,NaN,NaN
56,Cook Islands,New Zealand Dollar,NZD,NaN,NaN


In [1212]:
curr = curr.dropna(subset= ['Name'])
curr = curr.drop(['Name'], axis=1)

#### Check language - country

In [1213]:
lang = lang.merge(country[['Name','Iso3']], how='left', left_on='Country', right_on='Name')
lang.head()

,Country,Official Language,Name,Iso3
0,Afghanistan,"Dari Persian, Pashtu",Afghanistan,AFG
1,Albania,Albanian,Albania,ALB
2,Algeria,Arabic,Algeria,DZA
3,Andorra,Catalã¡N,Andorra,ADO
4,Angola,Portuguese,Angola,AGO


In [1214]:
lang[lang['Name'].isnull()]

,Country,Official Language,Name,Iso3
50,East Timor,"Tetum, Portuguese",NaN,NaN
116,Micronesia,English,NaN,NaN
125,Nauru,Nauruan,NaN,NaN
136,Palestinian State (Proposed),"Arabic, Hebrew, English",NaN,NaN
153,Sã£O Tomã© E Princã­Pe,Portuguese,NaN,NaN
193,Vatican City (Holy See),"Italian, Latin, French, Various Other Languages",NaN,NaN
196,Western Sahara (Proposed State),"Hassaniya Arabic, Moroccan Arabic",NaN,NaN


In [1215]:
lang = lang.dropna(subset=['Name'])
lang = lang.drop(['Name'], axis=1)

#### Check peace - country

In [1216]:
peace=peace.merge(country[['Name','Iso3']], how='left', left_on='Country', right_on='Name')
peace.head()

,Rank,Country,Name,Iso3
0,1.0,Iceland,Iceland,ISL
1,2.0,New Zealand,New Zealand,NZL
2,3.0,Denmark,Denmark,DNK
3,4.0,Portugal,Portugal,PRT
4,5.0,Slovenia,Slovenia,SVN


In [1217]:
peace[peace['Name'].isnull()]

,Rank,Country,Name,Iso3
52,53.0,The Gambia,NaN,NaN
68,69.0,Eswatini,NaN,NaN
138,138.0,Palestine,NaN,NaN
155,155.0,Central African\nRepublic,NaN,NaN


In [1218]:
peace=peace.dropna(subset=['Name'])
peace = peace.drop(['Name'], axis=1)

#### Check population - country

In [1219]:
pop=pop.merge(country[['Name','Iso3']], how='left', left_on='Country', right_on='Name')
pop.head()

,Country,Country Code,Pop_2021,Name,Iso3
0,Aruba,ABW,107195.0,Aruba,ABW
1,Africa Eastern And Southern,AFE,694665117.0,NaN,NaN
2,Afghanistan,AFG,39835428.0,Afghanistan,AFG
3,Africa Western And Central,AFW,470898870.0,NaN,NaN
4,Angola,AGO,33933611.0,Angola,AGO


In [1220]:
pop[pop['Name'].isnull()]

,Country,Country Code,Pop_2021,Name,Iso3
1,Africa Eastern And Southern,AFE,6.946651e+08,NaN,NaN
3,Africa Western And Central,AFW,4.708989e+08,NaN,NaN
7,Arab World,ARB,4.445178e+08,NaN,NaN
36,Central Europe And The Baltics,CEB,1.016696e+08,NaN,NaN
49,Caribbean Small States,CSS,7.481631e+06,NaN,NaN
61,East Asia & Pacific (Excluding High Income),EAP,2.122086e+09,NaN,NaN
62,Early-Demographic Dividend,EAR,3.373867e+09,NaN,NaN
63,East Asia & Pacific,EAS,2.368623e+09,NaN,NaN
64,Europe & Central Asia (Excluding High Income),ECA,4.018289e+08,NaN,NaN
65,Europe & Central Asia,ECS,9.237537e+08,NaN,NaN


In [1221]:
pop = pop.dropna(subset=['Name'])

In [1222]:
# drop both country name and iso code from the merged dataset, as population data already contains ISO code
pop = pop.drop(['Name', 'Iso3'], axis=1)

#### Check Religion - country

In [1223]:
rel=rel.merge(country[['Name','Iso3']], how='left', left_on='Country', right_on='Name')
rel.head()

,Country,Main Religion,Name,Iso3
0,Albania,Muslim,Albania,ALB
1,Anguilla,Anglican,NaN,NaN
2,Antigua And Barbuda,Anglican,Antigua And Barbuda,ATG
3,Armenia,Armenian Apostolic,Armenia,ARM
4,Aruba,Roman Catholic,Aruba,ABW


In [1224]:
rel[rel['Name'].isnull()]

,Country,Main Religion,Name,Iso3
1,Anguilla,Anglican,NaN,NaN
15,British Virgin Islands,Methodist,NaN,NaN
24,Cook Islands,Christian,NaN,NaN
32,Falkland Islands (Malvinas),Christian,NaN,NaN
38,Gibraltar,Roman Catholic,NaN,NaN
61,Micronesia (Federated States Of),Roman Catholic,NaN,NaN
64,Montserrat,Anglican,NaN,NaN
71,Niue,Ekalesia Niue,NaN,NaN
72,Norfolk Island,Church Of England,NaN,NaN
86,Saint Helena Ex. Dep.,Church Of England,NaN,NaN


In [1225]:
rel = rel.dropna(subset=['Name'])
rel = rel.drop(['Name'], axis=1)

#### Check Average Restaurant and Accomodation Cost - ISO

In [1226]:
avg_rest_hot_p = avg_rest_hot_p.merge(country[['Name','Iso3']], how='left', left_on='Country Code', right_on='Iso3')
avg_rest_hot_p.head()

,Country,Country Code,Average Hotel and Restaurant Cost,Type Traveler,Name,Iso3
0,Albania,ALB,50.78,Budget Traveler,Albania,ALB
1,Algeria,DZA,33.93,Budget Traveler,Algeria,DZA
2,American Samoa,ASM,NaN,Non Applicable,American Samoa,ASM
3,Angola,AGO,79.22,Comfort traveler,Angola,AGO
4,Anguilla,AIA,122.30,Comfort traveler,NaN,NaN


In [1227]:
avg_rest_hot_p [avg_rest_hot_p ['Name'].isnull()]

,Country,Country Code,Average Hotel and Restaurant Cost,Type Traveler,Name,Iso3
4,Anguilla,AIA,122.30,Comfort traveler,NaN,NaN
28,Bonaire,BON,156.57,Luxury Travel,NaN,NaN
62,Cook Islands,COK,NaN,Non Applicable,NaN,NaN
78,East Asia & Pacific (Icp),EAB,80.57,Comfort traveler,NaN,NaN
88,Europe & Central Asia (Icp),ECB,120.78,Comfort traveler,NaN,NaN
150,Latin America & Caribbean (Icp),LCB,78.35,Comfort traveler,NaN,NaN
176,Middle East & North Africa (Icp),MEB,64.50,Budget Traveler,NaN,NaN
183,Montserrat,MSR,84.58,Comfort traveler,NaN,NaN
189,Nauru,NRU,NaN,Non Applicable,NaN,NaN
199,Niue,NIU,NaN,Non Applicable,NaN,NaN


In [1228]:
avg_rest_hot_p = avg_rest_hot_p.dropna(subset=['Name'])
# as the dataset contains both country name and country code, delete the merged columns
avg_rest_hot_p = avg_rest_hot_p.drop(['Name', 'Iso3'], axis=1)

#### Cuisine - Country

In [1229]:
cuisine_temp = cuisine_rank.merge(country[['Name','Iso3']], how='left', left_on='Country', right_on='Name')
cuisine_temp.head()

,Country,Ratings,Name,Iso3
0,Italy,4.72,Italy,ITA
1,Greece,4.69,Greece,GRC
2,Spain,4.59,Spain,ESP
3,Japan,4.59,Japan,JPN
4,India,4.54,India,IND


In [1230]:
cuisine_temp[cuisine_temp['Name'].isnull()]

,Country,Ratings,Name,Iso3
28,England,4.18,NaN,NaN
37,Bih,3.99,NaN,NaN
41,Palestine,3.98,NaN,NaN
68,Northern Ireland,3.91,NaN,NaN
79,Wales,3.88,NaN,NaN
87,Scotland,3.82,NaN,NaN


As the United Kingdom was divided to England, Northern Ireland, Wales and Scotland,
we will have one row for the United Kingdom with the average ratings of the 4 countries
as we have the United Kingdom in the WEF datset.

In [1231]:
# add average of 4 nations in the United Kingdom to the existing row
uk_rank = cuisine_rank.iloc[[28,68,79,87]]['Ratings'].mean()
cuisine_rank.loc[len(cuisine_rank.index)] = ['United Kingdom', uk_rank]

In [1232]:
# Then sort by ratings, and reset index
cuisine_rank.sort_values(by=['Ratings'], ignore_index=True, inplace=True, ascending=False)

In [1233]:
# merge again with modified data
cuisine_rank = cuisine_rank.merge(country[['Name','Iso3']], how='left', left_on='Country', right_on='Name')
# check UK
cuisine_rank[cuisine_rank['Country']=='United Kingdom']

,Country,Ratings,Name,Iso3
49,United Kingdom,3.9475,United Kingdom,GBR


In [1234]:
cuisine_rank[cuisine_rank['Name'].isnull()]

,Country,Ratings,Name,Iso3
28,England,4.18,NaN,NaN
37,Bih,3.99,NaN,NaN
41,Palestine,3.98,NaN,NaN
64,Northern Ireland,3.91,NaN,NaN
79,Wales,3.88,NaN,NaN
87,Scotland,3.82,NaN,NaN


In [1235]:
cuisine_rank = cuisine_rank.dropna(subset=['Name'])
cuisine_rank = cuisine_rank.drop(['Name'], axis=1)

#### LGBT data - Country

In [1236]:
lgbtq = lgbtq.merge(country[['Name', 'Iso3']], how='left', left_on='COUNTRY', right_on='Name')
lgbtq.head()

,RANK,COUNTRY,GAI,Name,Iso3
0,1,Iceland,9.78,Iceland,ISL
1,2,Netherlands,9.46,Netherlands,NLD
2,3,Norway,9.38,Norway,NOR
3,4,Sweden,9.18,Sweden,SWE
4,5,Canada,9.02,Canada,CAN


In [1237]:
lgbtq[lgbtq['Name'].isnull()]

,RANK,COUNTRY,GAI,Name,Iso3
44,45,Northern Cyprus,5.59,NaN,NaN
129,130,Palestine,3.20,NaN,NaN
157,158,Nagorno- Karabakh,2.54,NaN,NaN


In [1238]:
lgbtq=lgbtq.dropna(subset=['Name'])
lgbtq = lgbtq.drop(['Name'], axis=1)

In [1239]:
country.to_csv('../cleaned_data/country.csv')

pop.to_csv('../cleaned_data/population.csv')

clim.to_csv('../cleaned_data/climate.csv')

curr.to_csv('../cleaned_data/currencies.csv')

lang.to_csv('../cleaned_data/language.csv')

rel.to_csv('../cleaned_data/religion.csv')

peace.to_csv('../cleaned_data/peace_index.csv')

wef.to_csv('../cleaned_data/wef_ttdi.csv')

avg_rest_hot_p.to_csv("../cleaned_data/avg_rest_hot_p.csv")

lgbtq.to_csv("../cleaned_data/lgbtq.csv")

cuisine_rank.to_csv('../cleaned_data/cuisine_rank.csv')
